In [25]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [26]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [27]:
from schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [28]:
from typing import Any


async def dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [29]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=[User.name],
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    match_by=[User.name],
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                        "organization_countries": Organization.ccs,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [30]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "organization_countries": ["USA", "Mexico", "Canada"],
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "organization_countries": ["USA", "Mexico"],
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "organization_countries": ["USA", "Canada"],
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [31]:
# from pathlib import Path
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(
      schema=TestSchema, 
      # url=Path("./test.xlsx")
    )
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

In [32]:
db.describe()


{
    'schema': {
        'package': 'py-research',
        'module': 'schema',
        'object': 'TestSchema',
        'package_version': '^5.1.1.dev148+g44d4d92.d20250123',
        'repo': 'https://github.com/cloudlane-one/py-research.git',
        'repo_revision': 'da00ea85acc91437b62034533f9c4372decab135',
        'module_path': WindowsPath('exp/schema.py'),
        'module_dirty': False,
        'docs_url': 'https://cloudlane-one.github.io/py-research/'
    },
    'contents': {
        'records': {
            'schema.Search': 1,
            'schema.Organization': 3,
            'schema.Membership': 7,
            'schema.Project': 3,
            'schema.User': 3,
            'schema.Task': 6,
            'schema.SearchResult': 3
        },
        'arrays': {'schema_Organization_ccs': 8},
        'relations': {'schema.Membership': 7, 'schema.SearchResult': 3}
    },
    'backend': None
}

In [33]:
nodes, edges = db.to_graph([Search, Project, Organization])

In [34]:
nodes

,node_id,schema_Search_term,schema_Search_result_count,table,schema_Project_number,schema_Project_org__id,schema_Project_name,schema_Project_end,schema_Project_start,schema_Project_status,schema_Organization__id,schema_Organization_address,schema_Organization_name,schema_Organization_city
0,0,test,3.0,schema_Search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,schema_Project,0.0,f18a7b0e-4828-47f7-9b91-126cb3f9af2f,baking cake,2020-01-04,2020-01-01,done,NaN,NaN,NaN,NaN
2,2,NaN,NaN,schema_Project,1.0,79ba7912-a9fc-4379-b2d5-cfb3ebbd9b4e,cleaning shoes,2020-01-05,2020-01-02,done,NaN,NaN,NaN,NaN
3,3,NaN,NaN,schema_Project,2.0,8ba4de5f-fdad-4051-9185-c80277cd1c98,fixing cars,2020-01-06,2020-01-03,started,NaN,NaN,NaN,NaN
4,4,NaN,NaN,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,f18a7b0e-4828-47f7-9b91-126cb3f9af2f,Main Street 1,Bakery,Bakerville
5,5,NaN,NaN,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,8ba4de5f-fdad-4051-9185-c80277cd1c98,Main Street 2,Shoe Shop,Shoetown
6,6,NaN,NaN,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,79ba7912-a9fc-4379-b2d5-cfb3ebbd9b4e,Main Street 3,Car Shop,Cartown


In [35]:
edges

,source,target,ltr,rtl,schema_SearchResult__to_number,schema_SearchResult_score,schema_SearchResult__id,schema_SearchResult__from_term
0,1,4,org__id,None,NaN,NaN,NaN,NaN
1,2,6,org__id,None,NaN,NaN,NaN,NaN
2,3,5,org__id,None,NaN,NaN,NaN,NaN
3,0,1,_to_number,_from_term,0.0,0.8,2486015738,test
4,0,3,_to_number,_from_term,2.0,0.9,5271218892,test
5,0,2,_to_number,_from_term,1.0,0.7,3910210571,test


In [36]:
db[User].values()


[
    User({'_id': 'ba13df4c-0a32-4b0d-b367-d0f9c1f2c5c6', 'name': 'John', 'age': 31}),
    User({'_id': '4e9a1dd1-3a18-4f8b-b95c-37652aa238d2', 'name': 'Jane', 'age': 56}),
    User({'_id': '4c8b4002-7a0a-48a4-a624-5b64f3160562', 'name': 'Jack', 'age': 66})
]

In [37]:
db[Search.term].values()

['test']

In [38]:
list(db[Search.result_count].items())

[('test', 3)]

In [39]:
list(db[Search.results.x.org].items())


[
    (
        ('test', 'f18a7b0e-4828-47f7-9b91-126cb3f9af2f', 0),
        Organization({'_id': 'f18a7b0e-4828-47f7-9b91-126cb3f9af2f', 'name': 'Bakery', 'address': 'Main Street 1', 'city': 'Bakerville'})
    ),
    (
        ('test', '8ba4de5f-fdad-4051-9185-c80277cd1c98', 2),
        Organization({'_id': '8ba4de5f-fdad-4051-9185-c80277cd1c98', 'name': 'Shoe Shop', 'address': 'Main Street 2', 'city': 'Shoetown'})
    ),
    (
        ('test', '79ba7912-a9fc-4379-b2d5-cfb3ebbd9b4e', 1),
        Organization({'_id': '79ba7912-a9fc-4379-b2d5-cfb3ebbd9b4e', 'name': 'Car Shop', 'address': 'Main Street 3', 'city': 'Cartown'})
    )
]

In [40]:
list(db[Search.results].items())[0][1].start.year

2020

In [41]:
list(db[Search.results]._abs_cols.keys())

['number', 'name', 'start', 'end', 'status', 'org__id']

In [42]:
db[Search.results].df()

schema_Search_term,start,end,status,number,name,org__id
str,date,date,str,i64,str,str
"""test""",2020-01-01,2020-01-04,"""done""",0,"""baking cake""","""f18a7b0e-4828-47f7-9b91-126cb3…"
"""test""",2020-01-03,2020-01-06,"""started""",2,"""fixing cars""","""8ba4de5f-fdad-4051-9185-c80277…"
"""test""",2020-01-02,2020-01-05,"""done""",1,"""cleaning shoes""","""79ba7912-a9fc-4379-b2d5-cfb3eb…"


In [43]:
db[Search.results.x.org].df()

_id,schema_Search_results_number,name,address,city,schema_Search_term
str,i64,str,str,str,str
"""f18a7b0e-4828-47f7-9b91-126cb3…",0,"""Bakery""","""Main Street 1""","""Bakerville""","""test"""
"""8ba4de5f-fdad-4051-9185-c80277…",2,"""Shoe Shop""","""Main Street 2""","""Shoetown""","""test"""
"""79ba7912-a9fc-4379-b2d5-cfb3eb…",1,"""Car Shop""","""Main Street 3""","""Cartown""","""test"""


In [44]:
db[Organization].df()

_id,name,city,address
str,str,str,str
"""f18a7b0e-4828-47f7-9b91-126cb3…","""Bakery""","""Bakerville""","""Main Street 1"""
"""8ba4de5f-fdad-4051-9185-c80277…","""Shoe Shop""","""Shoetown""","""Main Street 2"""
"""79ba7912-a9fc-4379-b2d5-cfb3eb…","""Car Shop""","""Cartown""","""Main Street 3"""


In [45]:
db[SearchResult._table[SearchResult.score.isin(slice(0.81, 0.9))]].df()

_to_number,_id,score,_from_term
i64,str,f64,str
2,"""5271218892""",0.9,"""test"""


In [46]:
import pandas as pd


db[Organization.ccs].df(pd.DataFrame)

value
schema_Organization__id              schema_Organization_ccs_idx        
f18a7b0e-4828-47f7-9b91-126cb3f9af2f 0                               USA
                                     1                            Mexico
                                     2                            Canada
79ba7912-a9fc-4379-b2d5-cfb3ebbd9b4e 0                               USA
                                     1                            Canada
8ba4de5f-fdad-4051-9185-c80277cd1c98 0                               USA
                                     1                            Mexico
                                     2                            Canada

In [47]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         387482 function calls (360898 primitive calls) in 0.692 seconds

   Ordered by: cumulative time
   List reduced from 1800 to 153 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       52    0.000    0.000    0.263    0.005 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:697(_load_rec_from_item)
       52    0.001    0.000    0.263    0.005 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:572(_load_record)
       19    0.000    0.000    0.240    0.013 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1496(__ior__)
       19    0.001    0.000    0.239    0.013 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2958(_mutate)
       56    0.000    0.000    0.220    0.004 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:838(load)
   168/57    0.001    0.000    0.203    0.004 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py